In [1]:
using Distributions
using Random

function simordstat(dist, n, k; L = 10^6)
    X = Vector{Float64}(undef, L)
    tmp = [Vector{Float64}(undef, n) for _ in 1:Threads.nthreads()]
    Threads.@threads for i in 1:L
        X[i] = sort!(rand!(dist, tmp[Threads.threadid()]))[k]
    end
    X
end

epmf(X, a) = count(==(a), X)/length(X)

function pmfordstat_beta(dist, n, k, x)
    beta = Beta(k, n-k+1)
    cdf(beta, cdf(dist, x)) - cdf(beta, cdf(dist, x-1))
end

function pmfordstat_direct1(dist, n, k, x)
    beta = Beta(k, n-k+1)
    pdf(beta, cdf(dist, x)) * pdf(dist, x)
end

function pmfordstat_direct2(dist, n, k, x)
    beta = Beta(k, n-k+1)
    factorial(n)/(factorial(k-1)*factorial(n-k)) * cdf(dist, x)^(k-1) * pdf(dist, x) * (1 - cdf(dist, x))^(n-k)
end

pmfordstat_direct2 (generic function with 1 method)

In [2]:
dist, n, k = Binomial(10, 0.3), 9, 5
@time X = simordstat(dist, n, k; L=10^7)
x = support(dist)
a = epmf.(Ref(X), x)
b = pmfordstat_beta.(dist, n, k, x)
c = pmfordstat_direct1.(dist, n, k, x)
d = pmfordstat_direct2.(dist, n, k, x)
[a b c d]

  1.183266 seconds (538.79 k allocations: 104.969 MiB, 0.89% gc time, 23.52% compilation time)


11×4 Matrix{Float64}:
 1.3e-6     2.06032e-6  1.01033e-5   1.01033e-5
 0.0055013  0.00551232  0.0198504    0.0198504
 0.226239   0.226141    0.458291     0.458291
 0.595847   0.595967    0.451219     0.451219
 0.166717   0.166703    0.0335143    0.0335143
 0.0056683  0.00564794  0.00026842   0.00026842
 2.52e-5    2.55177e-5  2.79322e-7   2.79322e-7
 0.0        1.62124e-8  3.60504e-11  3.60504e-11
 0.0        1.2752e-12  3.88263e-16  3.88263e-16
 0.0        0.0         1.05529e-22  1.05529e-22
 0.0        0.0         0.0          0.0

In [3]:
dist, n, k = Poisson(3), 11, 5
@time X = simordstat(dist, n, k; L=10^7)
@show x = 0:10
a = epmf.(Ref(X), x)
b = pmfordstat_beta.(dist, n, k, x)
c = pmfordstat_direct1.(dist, n, k, x)
d = pmfordstat_direct2.(dist, n, k, x)
[a b c d]

  0.864669 seconds (338.65 k allocations: 94.345 MiB, 1.28% gc time, 15.61% compilation time)
x = 0:10 = 0:10


11×4 Matrix{Float64}:
 0.0001109  0.000109688  0.00052014   0.00052014
 0.0494444  0.0494791    0.143175     0.143175
 0.481433   0.481384     0.611335     0.611335
 0.416719   0.416755     0.175032     0.175032
 0.0510872  0.0510717    0.00681575   0.00681575
 0.0011974  0.00119422   5.72822e-5   5.72822e-5
 7.9e-6     7.11917e-6   1.4383e-7    1.4383e-7
 0.0        1.3887e-8    1.35397e-10  1.35397e-10
 0.0        1.0719e-11   5.57586e-14  5.57586e-14
 0.0        3.77476e-15  1.11528e-17  1.11528e-17
 0.0        0.0          1.16674e-21  1.16674e-21

In [4]:
dist, n, k = Hypergeometric(10, 10, 10), 11, 5
@time X = simordstat(dist, n, k; L=10^7)
@show x = support(dist)
a = epmf.(Ref(X), x)
b = pmfordstat_beta.(dist, n, k, x)
c = pmfordstat_direct1.(dist, n, k, x)
d = pmfordstat_direct2.(dist, n, k, x)
[a b c d]

  1.704890 seconds (115.64 k allocations: 82.540 MiB, 0.57% gc time, 2.87% compilation time)
x = support(dist) = 0:10


11×4 Matrix{Float64}:
 0.0        2.14604e-24  1.07301e-23  1.07301e-23
 0.0        2.24941e-14  1.11296e-13  1.11296e-13
 0.0        8.79782e-8   4.14132e-7   4.14132e-7
 0.0016759  0.00166665   0.00656874   0.00656874
 0.274431   0.274434     0.58798      0.58798
 0.688435   0.68842      0.201971     0.201971
 0.0354466  0.035468     0.000194127  0.000194127
 1.16e-5    1.09199e-5   3.99079e-10  3.99079e-10
 0.0        8.46634e-12  6.74257e-19  6.74257e-19
 0.0        0.0          3.14349e-32  3.14349e-32
 0.0        0.0          0.0          0.0